**Note about running apachebeam template in dataflow**
- dataflow is a serverless service to run data etl job. Therefore you do not need to set up server, we just need to send request with some configuration
- reference link: https://github.com/GoogleCloudPlatform/python-docs-samples/tree/473749daea6b1f1d5a6a8826093a970f03ce0517/dataflow/run_template
- content: discuss about how run apachebeam template in dataflow

**Before you begin**
- make sure we have:
    - cloud storage bucket
    - source and installed virtual environment wit installed google-api-python-client
- there are multiple ways to communicate with google cloud services
    - using cloud api (recommend ways)
    - using python client package
- reference for dataflow api (api is general documents for all languages:): https://cloud.google.com/dataflow/docs/reference/rest/v1b3/projects.templates/launch

**Note about how to map between api document and python code**
- end point: https://cloud.google.com/dataflow/docs/reference/rest/v1b3/projects.templates/launch. dataflow.projects.templates.launch
- python code: dataflow.prjects().templates().launch()
- beside normal path parameter (required) and query parameter (optional), we have special parameter body which is request body for api request
- on each end point documents we have request body format link and response body format link


In [ ]:
#package for google cloud client api: google-api-python-client
#example of using google-api-python-client
#end point: dataflow.projects.templates.launch.
#end point reference: https://cloud.google.com/dataflow/docs/reference/rest/v1b3/projects.templates/launch
#python call: dataflow.projects().templates().launch()
#it is good practice to read api end point document and request body, response body document
#note: in this example, we already have temmplate
from googleapiclient.discovery import build
dataflow = build('dataflow', 'v1b3')
request = dataflow.projects().templates().launch(
    projectId="project",
    gcsPath="template",
    body={
        'jobName': "job",
        'parameters': {"key_one": "value_one", "key_two": "value_two"},
    }
)

In [ ]:
#sample code to prepare environment and code for this exercise:
#for cloud storage related job we have gcloud storage vs gsutil cli. gsutils more detail for cloud storage task
#- reference for gsutils: https://cloud.google.com/storage/docs/gsutil
#- reference for gcloud cli: https://cloud.google.com/sdk/gcloud/reference
#note about structure of google cloud product docs page: 
    #- each product docs page we have tabs like: overview, guides, references, samples, support, resources
    # we care about reference (docs page), guides and samples
#note about api reference:
    #api document reference is not specific for any language. it is general interface for all language.
    #google cloud api documents page is also organize as tabs: overview, reference (docs), guides, samples

In [ ]:
#code sample for run dataflow template
#make cloud storage bucket with gsutils mb command
!export BUCKET=your-gcs-bucket
!gsutil mb gs://$BUCKET
#clone code with git clone command
!git clone https://github.com/GoogleCloudPlatform/python-docs-samples.git
#change directory with cd command
!cd python-docs-samples/dataflow/run_template
#make and activate virtual environment, to isolate vironment (python version and package)
!python3 -m venv venv
!source venv/bin/activate
#install requirement packages
!pip install -r requirements.txt

In [ ]:
#given that we have
PROJECT=$(gcloud config get-value project) \
REGION=$(gcloud config get-value functions/region)

# Deploy the Cloud Function. this will make cloud function running and the background and waiting for http request to trigger function
!gcloud functions deploy run_template \
  --runtime python37 \
  --trigger-http \
  --region $REGION

# Call the Cloud Function via an HTTP request. Every time we want to trigger function make a post request
!curl -X POST "https://$REGION-$PROJECT.cloudfunctions.net/run_template" \
  -d project=$PROJECT \
  -d job=wordcount-$(date +'%Y%m%d-%H%M%S') \
  -d template=gs://dataflow-templates/latest/Word_Count \
  -d inputFile=gs://apache-beam-samples/shakespeare/kinglear.txt \
  -d output=gs://<your-gcs-bucket>/wordcount/outputs

**Dataflow classic templates vs flex templates**
- classic template: is apache beam pipeline as code
- flex template: is apache beam pipeline but containerize

**Type of api request**
- GET: to get information
- POST: to make request resource with information 
- PUT: to update resource with request body
- DELETE: to delete resource resource